In [ ]:
import importlib
import subprocess
import sys
import gc

def check_and_install_package(package_name, version=None):
    try:
        importlib.import_module(package_name)
        print(f"\n{package_name} is already installed.")
    except ImportError:
        print(f"\n{package_name} is NOT installed. Installing now...")
        if version:
            subprocess.check_call([sys.executable, "-m", "pip", "install", f"{package_name}=={version}"])
        else:
            subprocess.check_call([sys.executable, "-m", "pip", "install", package_name])
        print(f"{package_name} installation completed.")

# List of packages to check along with specific versions if necessary
packages = [
    {"name": "tqdm", "version": None},
    {"name": "pyspark", "version": "3.5.2"},
    {"name": "gdown", "version": None},
    {"name": "numpy", "version": "1.23.5"}
]

# Checking and installing the packages
for package in packages:
    check_and_install_package(package["name"], package["version"])


tqdm is already installed.

pyspark is NOT installed. Installing now...
pyspark installation completed.

gdown is already installed.

numpy is already installed.


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
from pyspark.sql import SparkSession

spark = SparkSession.builder \
    .appName("GBTModel") \
    .config("spark.driver.memory", "16g") \
    .config("spark.executor.memory", "16g") \
    .config("spark.driver.maxResultSize", "8g") \
    .config("spark.executor.memoryOverhead", "12g") \
    .config("spark.executor.cores", "5") \
    .config("spark.serializer", "org.apache.spark.serializer.KryoSerializer") \
    .config("spark.kryoserializer.buffer.max", "2047m") \
    .config("spark.dynamicAllocation.enabled", "true") \
    .config("spark.hadoop.fs.file.impl", "org.apache.hadoop.fs.LocalFileSystem") \
    .config("spark.executor.extraJavaOptions", "-XX:+UseG1GC -XX:InitiatingHeapOccupancyPercent=35 -XX:ConcGCThreads=4 -XX:ParallelGCThreads=4") \
    .getOrCreate()

# Verifying Spark session creation
print(f"Spark session started with version: {spark.version}")


Spark session started with version: 3.5.2


In [ ]:
# loading the df

!cp '/content/drive/MyDrive/Big Data Analytics - Project/Datasets/Processed_DF.parquet' /content/

output_path = '/content/Processed_DF.parquet'
df = spark.read.parquet(output_path)
print("The Processed DataFrame has been loaded successfully.")


The Processed DataFrame has been loaded successfully.


In [ ]:
df = df.repartition(100)  # Repartitioning into 100 partitions for parallelism

In [ ]:
# Printing the shape of the DataFrame
total_rows = df.count()
total_columns = len(df.columns)

print(f"The shape of the loaded DataFrame is: ({total_rows}, {total_columns})")

The shape of the loaded DataFrame is: (3000040, 42)


In [ ]:
from functools import reduce
from pyspark.sql.functions import col

# Checking for rows with at least one null value in any column
rows_with_null = df.filter(
    reduce(lambda x, y: x | y, (col(c).isNull() for c in df.columns))).count()

print(f"Checking for Null values -> Number of rows with at least one null value: {rows_with_null}")

Checking for Null values -> Number of rows with at least one null value: 0


### **Handling Categorical Coloumns**

In [ ]:
df=df.drop('description','major_options','mileage')
# Keeping the columns ['exterior_color','dealer_zip','interior_color']

In [ ]:
# Counting unique values in 'exterior_color' and 'interior_color' columns
exterior_colors_count = df.select('exterior_color').distinct().count()
interior_colors_count = df.select('interior_color').distinct().count()

print(f"Unique exterior colors: {exterior_colors_count}")
print(f"Unique interior colors: {interior_colors_count}")

Unique exterior colors: 23036
Unique interior colors: 38528


In [ ]:
from pyspark.sql import functions as F
from pyspark.sql.types import ArrayType, StringType

# Listing top colors for both exterior and interior
colors = ['White', 'Black', 'Gray', 'Silver', 'Red', 'Blue', 'Brown', 'Green', 'Beige', 'Orange', 'Gold', 'Yellow', 'Purple']

# Creating a UDF to find colors in the color columns
@F.udf(returnType=ArrayType(StringType()))
def find_colors(color_string):
    if color_string is None or color_string.strip() == "":
        return ["Other"]  # Handle empty or null values
    found_colors = [c for c in colors if c.lower() in color_string.lower()]
    return found_colors if found_colors else ["Other"]  # Label non-matching colors as "Other"

# Applying the UDF to both the exterior and interior color columns
df = df.withColumn("exterior_color_array", find_colors("exterior_color"))
df = df.withColumn("interior_color_array", find_colors("interior_color"))

# Creating a column with the count of colors found for both exterior and interior
df = df.withColumn("exterior_color_count", F.size("exterior_color_array"))
df = df.withColumn("interior_color_count", F.size("interior_color_array"))

# Joining the color arrays into string columns
df = df.withColumn("exterior_color", F.array_join("exterior_color_array", ", "))
df = df.withColumn("interior_color", F.array_join("interior_color_array", ", "))

# Labeling mixed colors for both exterior and interior colors
df = df.withColumn(
    "exterior_color",
    F.when(F.col("exterior_color_count") > 1, "Mixed Colors")
     .otherwise(F.col("exterior_color")))

df = df.withColumn(
    "interior_color",
    F.when(F.col("interior_color_count") > 1, "Mixed Colors")
     .otherwise(F.col("interior_color")))

# Dropping temporary columns
df = df.drop("exterior_color_array", "exterior_color_count", "interior_color_array", "interior_color_count")


In [ ]:
# Counting the occurrences of each exterior and interior color and calculating percentages
exterior_color_counts = df.groupBy("exterior_color").count().withColumn(
    "percentage", F.round((F.col("count") / df.count()) * 100, 2))

interior_color_counts = df.groupBy("interior_color").count().withColumn(
    "percentage", F.round((F.col("count") / df.count()) * 100, 2))


# Showing the results
print("Exterior Color Distribution:")
exterior_color_counts.orderBy(F.desc("count")).show(truncate=False)

Exterior Color Distribution:
+--------------+------+----------+
|exterior_color|count |percentage|
+--------------+------+----------+
|White         |675979|22.53     |
|Black         |580148|19.34     |
|Other         |563366|18.78     |
|Silver        |384540|12.82     |
|Blue          |253263|8.44      |
|Red           |242331|8.08      |
|Gray          |231172|7.71      |
|Green         |23026 |0.77      |
|Brown         |12905 |0.43      |
|Orange        |11638 |0.39      |
|Gold          |10544 |0.35      |
|Beige         |5065  |0.17      |
|Yellow        |4855  |0.16      |
|Purple        |1208  |0.04      |
+--------------+------+----------+



In [ ]:
print("Interior Color Distribution:")
interior_color_counts.orderBy(F.desc("count")).show(truncate=False)

Interior Color Distribution:
+--------------+-------+----------+
|interior_color|count  |percentage|
+--------------+-------+----------+
|Black         |1624033|54.13     |
|Other         |748790 |24.96     |
|Gray          |383966 |12.8      |
|White         |91545  |3.05      |
|Brown         |65943  |2.2       |
|Red           |34117  |1.14      |
|Silver        |24124  |0.8       |
|Blue          |22828  |0.76      |
|Green         |2048   |0.07      |
|Gold          |1193   |0.04      |
|Orange        |1133   |0.04      |
|Yellow        |134    |0.0       |
|Purple        |121    |0.0       |
|Beige         |65     |0.0       |
+--------------+-------+----------+



In [ ]:
print(f"Final processed DataFrame used for the model has {df.count()} rows and {len(df.columns)} columns.")

Final processed DataFrame used for the model has 3000040 rows and 39 columns.


In [ ]:
# Calculating the average price
avg_price = df.agg({"price": "avg"}).collect()[0][0]
print(f"Average price of a car: {round(avg_price)}")

Average price of a car: 29933


In [ ]:
import pandas as pd
from IPython.display import display
import pyspark.sql.functions as F

# Converting the Spark DataFrame to a Pandas DataFrame and displaying 5 random rows with all columns
pd.set_option('display.max_columns', None)
pandas_df = df.orderBy(F.rand()).limit(5).toPandas()
display(pandas_df)


,fuel_type,body_type,city,city_fuel_economy,daysonmarket,dealer_zip,engine_displacement,engine_type,exterior_color,franchise_dealer,fuel_tank_volume,height,highway_fuel_economy,horsepower,interior_color,is_new,latitude,length,listed_date,listing_color,longitude,make_name,maximum_seating,model_name,price,savings_amount,seller_rating,sp_name,torque,transmission,transmission_display,wheel_system_display,wheelbase,width,year,combined_fuel_economy,legroom,log_mileage,major_options_count
0,Gasoline,SUV / Crossover,Nashville,27.000000,64,37228,1500.0,I4,White,True,14.00,66.50,32.000000,190.0,White,True,36.193199,182.1,2020-07-09,GRAY,-86.803596,Honda,5.0,CR-V,25161.0,0,4.333333,Crest Honda,179.00,CVT,Continuously Variable Transmission,All-Wheel Drive,104.7,73.0,2020,29.50,81.70,8.91,0
1,Gasoline,SUV / Crossover,Palmyra,25.000000,344,17078,1500.0,I4,Silver,True,15.60,65.40,30.000000,170.0,Black,False,40.311901,183.1,2019-10-02,SILVER,-76.586800,Chevrolet,5.0,Equinox,21506.0,1145,3.909091,Klick Lewis,265.22,A,Automatic,Four-Wheel Drive,107.3,72.6,2019,27.50,80.80,9.53,5
2,Gasoline,Sedan,Saint Peters,22.690001,1,63376,2500.0,Gasoline engine,Other,True,18.09,65.87,29.469999,241.0,Other,True,38.798100,189.8,2020-09-10,UNKNOWN,-90.599098,Hyundai,5.0,Sonata,33067.0,0,4.258065,St. Charles Automotive,265.22,A,Automatic,Unknown,111.0,77.2,2021,26.08,80.16,1.95,0
3,Diesel,Pickup Truck,Thurmont,22.690001,154,21788,6700.0,I6,Other,True,32.00,79.30,29.469999,370.0,Black,True,39.617298,260.8,2020-04-09,GRAY,-77.416397,RAM,6.0,3500,72053.0,0,4.487805,Criswell Chrysler Dodge Jeep Ram,850.00,A,6-Speed Automatic,Four-Wheel Drive,169.0,96.5,2020,26.08,81.10,0.69,12
4,Gasoline,SUV / Crossover,Bluffton,16.000000,43,29910,3500.0,V6,White,True,28.00,77.20,22.000000,365.0,Other,False,32.255901,206.0,2020-07-29,WHITE,-80.848900,Ford,8.0,Expedition,19964.0,1845,4.375000,Hilton Head Cadillac Buick GMC,420.00,A,6-Speed Automatic,4X2,119.0,91.8,2015,19.00,82.10,11.99,12


# **GBT Regressor**

**DEMO Code for checking if libraries are properly installed**

In [ ]:
from pyspark.ml.regression import GBTRegressor
from pyspark.ml.linalg import Vectors
from pyspark.sql import Row
from pyspark.ml.evaluation import RegressionEvaluator

# Sample data creation: creating a simple DataFrame with features and a label for regression
data1 = [
    Row(label=1.0, features=Vectors.dense([1.0, 2.0, 3.0])),
    Row(label=2.0, features=Vectors.dense([4.0, 5.0, 6.0])),
    Row(label=3.0, features=Vectors.dense([7.0, 8.0, 9.0])),
    Row(label=4.0, features=Vectors.dense([10.0, 11.0, 12.0]))
]

df1 = spark.createDataFrame(data1)

# Splitting the data into train and test sets
train_df1, test_df1 = df1.randomSplit([0.8, 0.2], seed=42)

# Defining GBTRegressor
gbt_regressor = GBTRegressor(
    featuresCol="features",
    labelCol="label",
    maxIter=10,
    maxDepth=3
)

# Training the model
model = gbt_regressor.fit(train_df1)

# Making predictions
predictions = model.transform(test_df1)

# Showing predictions
predictions.select("features", "label", "prediction").show()

# Evaluating the model using RegressionEvaluator
evaluator = RegressionEvaluator(labelCol="label", predictionCol="prediction", metricName="rmse")
rmse = evaluator.evaluate(predictions)

print(f"\nRoot Mean Squared Error (RMSE): {rmse:.4f}")


+-------------+-----+----------+
|     features|label|prediction|
+-------------+-----+----------+
|[1.0,2.0,3.0]|  1.0|       2.0|
+-------------+-----+----------+


Root Mean Squared Error (RMSE): 1.0000




---



### **On 40% of the Dataset**

In [ ]:
import warnings
from tqdm import tqdm
from pyspark.ml.feature import StringIndexer, VectorAssembler, StandardScaler, OneHotEncoder
from pyspark.ml import Pipeline
from pyspark.sql.functions import mean as sql_mean
import pyspark.sql.functions as F

# Ignore warnings
warnings.filterwarnings('ignore')

print("Processing the data...")
with tqdm(total=5, desc="Progress") as pbar:

    df_sample = df.sample(fraction=0.4, seed=42)  # Random sampling 40% of the data
    pbar.update(1)

    # Handling categorical columns
    cat_columns = [field for (field, dtype) in df_sample.dtypes if dtype == "string"]
    stages = []
    for col_name in cat_columns:
        indexer = StringIndexer(inputCol=col_name, outputCol=f"{col_name}_indexed", handleInvalid="keep")
        encoder = OneHotEncoder(inputCol=f"{col_name}_indexed", outputCol=f"{col_name}_encoded")
        stages += [indexer, encoder]
    pbar.update(1)

    # Assembling features
    num_columns = [col for col in df_sample.columns if col != 'price' and col not in cat_columns]
    encoded_columns = [f"{col}_encoded" for col in cat_columns]
    feature_columns = num_columns + encoded_columns
    assembler = VectorAssembler(inputCols=feature_columns, outputCol="features")
    stages += [assembler]
    pbar.update(1)

    # Adding scaling to the pipeline
    scaler = StandardScaler(inputCol="features", outputCol="scaled_features", withMean=True, withStd=True)
    stages += [scaler]

    # Creating and applying the pipeline
    pipeline = Pipeline(stages=stages)
    pipeline_model = pipeline.fit(df_sample)
    df_sample = pipeline_model.transform(df_sample)
    pbar.update(1)

    # Splitting the data
    train_df, test_df = df_sample.randomSplit([0.8, 0.2], seed=42)
    pbar.update(1)

print("\n\nData preprocessing and splitting completed!")


Processing the data...


Progress: 100%|██████████| 5/5 [01:18<00:00, 15.75s/it]



Data preprocessing and splitting completed!


In [ ]:
from pyspark.ml.regression import GBTRegressor
from pyspark.ml.evaluation import RegressionEvaluator
import time

# Starting to track overall run time
start_time = time.time()

# Model training
print("Training GBTRegressor model...")

# Using GBTRegressor with tuned parameters
gbt_regressor = GBTRegressor(
    featuresCol="scaled_features",
    labelCol="price",
    maxIter=100,
    maxDepth=5,
    seed=42,
)

# Training the model
model = gbt_regressor.fit(train_df)

# Making predictions
print("Making predictions...")
predictions = model.transform(test_df)

# Evaluating the model
print("Evaluating the model...")
evaluator = RegressionEvaluator(labelCol="price", predictionCol="prediction", metricName="r2")
r2 = evaluator.evaluate(predictions)

print(f"\nTrain size: {train_df.count():,} samples")
print(f"Test size: {test_df.count():,} samples")

# Multiplying R-Squared by 100 and rounding it off
print(f"\n\nR-Squared Score (Accuracy): {round(r2 * 100)}%\n")

# Calculating total runtime
end_time = time.time()
total_runtime = (end_time - start_time) / 60
print(f"\nOverall runtime: {round(total_runtime)} minutes.")


Training GBTRegressor model...
Making predictions...
Evaluating the model...

Train size: 960,617 samples
Test size: 239,827 samples


R-Squared Score (Accuracy): 81%


Overall runtime: 480 minutes.


In [ ]:
# Calculating additional metrics
mae_evaluator = RegressionEvaluator(labelCol="price", predictionCol="prediction", metricName="mae")
mae = mae_evaluator.evaluate(predictions)

mse_evaluator = RegressionEvaluator(labelCol="price", predictionCol="prediction", metricName="mse")
mse = mse_evaluator.evaluate(predictions)

rmse_evaluator = RegressionEvaluator(labelCol="price", predictionCol="prediction", metricName="rmse")
rmse = rmse_evaluator.evaluate(predictions)

print("Additional Metrics:")
print(f"Mean Absolute Error: {round(mae)}")
print(f"Mean Squared Error: {round(mse)}")
print(f"Root Mean Squared Error: {round(rmse)}")


Additional Metrics:
Mean Absolute Error: 3550
Mean Squared Error: 70708545
Root Mean Squared Error: 8409


The Mean Absolute Error (`MAE`) of **\$3550**  suggests that, on average, the predicted car prices deviate from the actual prices by this amount. Given that the Mean car price is **\$29,933** , this error represents about `12.0% of the mean price`, which suggests that the model performs well.

In [ ]:
# Feature Importance (for GBT)
feature_importance = model.featureImportances
features_list = pipeline_model.stages[-2].getInputCols()  # Getting feature names from VectorAssembler

# Sorting features by their importance in descending order
sorted_features = [feature for feature, importance in sorted(zip(features_list, feature_importance), key=lambda x: x[1], reverse=True)]

# Printing ranked features from highest to lowest
print("Top 10 Features Ranked by Importance (Highest to Lowest):")
for rank, feature in enumerate(sorted_features[:10], 1):  # Limiting to top 10 features
    print(f"{rank}. {feature}")

Top 10 Features Ranked by Importance (Highest to Lowest):
1. horsepower
2. log_mileage
3. year
4. daysonmarket
5. torque
6. engine_displacement
7. major_options_count
8. city_fuel_economy
9. highway_fuel_economy
10. maximum_seating


In [ ]:
import pyspark

# Getting the list of all variables in the current notebook environment
for var_name in dir():
    # Checking if the variable is a Spark DataFrame
    if isinstance(eval(var_name), pyspark.sql.dataframe.DataFrame):
        # Deleting the variables
        del globals()[var_name]

# Performing garbage collection
import gc
_ = gc.collect()
print("All DataFrames have been deleted.")


spark.catalog.clearCache()
spark.stop()
print("Spark Stopped !")

All DataFrames have been deleted.
Spark Stopped !
